# Hierarchical LDA Demo

This notebook demonstrates how we can load the BBC Insight Dataset (http://mlg.ucd.ie/datasets/bbc.html), preprocess them via NLTK and run hierarchical LDA inference on the data. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os
basedir = '..'
sys.path.append(os.path.join(basedir))

import pylab as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
from hlda.sampler import HierarchicalLDA
from ipywidgets import widgets
from IPython.core.display import HTML, display

import string
import glob

## 1. Load test corpus

Load and preprocess text using NLTK. Below we load all tech articles from the BBC Insight corpus.

In [ ]:
stopset = stopwords.words('english') + list(string.punctuation) + ['will', 'also', 'said']

In [ ]:
    corpus = []
    all_docs = []
    vocab = set()

    stemmer = PorterStemmer()
    for filename in glob.glob(os.path.join(basedir,'bbc','tech','*.txt')):
        with open(filename) as f:
            try:

                doc = f.read().splitlines() 
                doc = filter(None, doc) # remove empty string
                doc = '. '.join(doc)

                to_remove = string.punctuation
                table = str.maketrans("", "", to_remove)
                doc = doc.translate(table) # strip punctuations

                to_remove = "0123456789"
                table = str.maketrans("", "", to_remove)
                doc = doc.translate(table) # strip numbers

                doc = doc.encode("utf8").decode('ascii', 'ignore') # ignore fancy unicode chars
                all_docs.append(doc)        

                tokens = word_tokenize(str(doc))
                filtered = []
                for w in tokens:
                    w = stemmer.stem(w.lower()) # use Porter's stemmer
                    if len(w) < 3:              # remove short tokens
                        continue
                    if w in stopset:            # remove stop words
                        continue
                    filtered.append(w)

                vocab.update(filtered)
                corpus.append(filtered)      

            except UnicodeDecodeError:
                print('Failed to load', filename)
                
print("Done.")

Create an inverted index for the words to position in the sorted vocab

In [ ]:
vocab = sorted(list(vocab))
vocab_index = {}
for i, w in enumerate(vocab):
    vocab_index[w] = i

Total number of documents in the corpus

In [ ]:
print(len(all_docs))

Total number of vocab. Also print the first 100 words in the sorted vocab.

In [ ]:
print(len(vocab))
print(", ".join([vocab[v] for v in np.random.randint(0, len(vocab), 100, dtype='int')]))

## 2. Visualise the data

Make some pretty word cloud using the Python Word Cloud package: https://github.com/amueller/word_cloud

In [ ]:
wordcloud = WordCloud(background_color='white').generate(' '.join(all_docs))
plt.figure(figsize=(12, 12))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## 3. Run hLDA

In [ ]:
print(len(vocab), len(corpus), len(corpus[0]), len(corpus[1]))

Convert words in the corpus into indices

In [ ]:
new_corpus = []
for doc in corpus:
    new_doc = []
    for word in doc:
        word_idx = vocab_index[word]
        new_doc.append(word_idx)
    new_corpus.append(new_doc)

In [ ]:
print(len(vocab), len(new_corpus))
print(corpus[0][0:10])
print(new_corpus[0][0:10])

Create hierarchical LDA object and run the sampler.

In [ ]:
n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 50   # the number of iterations between printing a brief summary of the topics so far
n_words = 5           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [ ]:
hlda = HierarchicalLDA(new_corpus, vocab, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

## 4. Visualise results

In [ ]:
colour_map = {
    0: 'blue',
    1: 'red',
    2: 'green'
}

def show_doc(d=0):
    
    node = hlda.document_leaves[d]
    path = []
    while node is not None:
        path.append(node)
        node = node.parent
    path.reverse()   
    
    n_words = 10
    with_weights = False    
    for n in range(len(path)):
        node = path[n]
        colour = colour_map[n] 
        msg = 'Level %d Topic %d: ' % (node.level, node.node_id)
        msg += node.get_top_words(n_words, with_weights)
        output = '<h%d><span style="color:%s">%s</span></h3>' % (n+1, colour, msg)
        display(HTML(output))
        
    display(HTML('<hr/><h5>Processed Document</h5>'))

    doc = corpus[d]
    output = ''
    for n in range(len(doc)):
        w = doc[n]
        l = hlda.levels[d][n]
        colour = colour_map[l]
        output += '<span style="color:%s">%s</span> ' % (colour, w)
    display(HTML(output))

If you run this notebook locally, you'd be able to flip through the documents in the corpus and see the topic assignments of individual words of the document...

#### No longer seems to work...

In [ ]:
widgets.interact(show_doc, d=(0, len(corpus)-1))

## 5. Dump the hlda object for further use later

https://stackoverflow.com/questions/18474791/decreasing-the-size-of-cpickle-objects

In [ ]:
import cPickle
import gzip

def save_zipped_pickle(obj, filename, protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)
        
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = cPickle.load(f)
        return loaded_object

In [ ]:
save_zipped_pickle(hlda, 'bbc_hlda.p')